### Define the constants

In [ ]:
GEN_DIR = "/content/GEN"
GT_DIR = "/content/GT"
DISTORTED_DIR  = "/content/DISTORTED"
BATCH_SIZE = 1

In [ ]:
new_size = (384, 512)

In [ ]:
image_path = "/content/GT/06648_00.png"

In [ ]:
image = Image.open(image_path).resize(new_size)
image.save(image_path)

In [ ]:
import os

input_folder = GEN_DIR
files = os.listdir(input_folder)

for file in files:
    # Check if the file is an image (you can add more image file extensions if needed)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        input_path = os.path.join(input_folder, file)

        # Open the image file
        original_image = Image.open(input_path)

        # Resize the image
        resized_image = original_image.resize(new_size)

        # Save the resized image, replacing the original file
        resized_image.save(input_path)
        print(f"Resized and replaced: {input_path}")

Resized and replaced: /content/GEN/00013_00_00013_00_0.jpg
Resized and replaced: /content/GEN/00428_00_00428_00_0.jpg
Resized and replaced: /content/GEN/00126_00_00126_00_0.jpg
Resized and replaced: /content/GEN/04559_00_04559_00_0.jpg
Resized and replaced: /content/GEN/00017_00_00017_00_0.jpg
Resized and replaced: /content/GEN/00145_00_00145_00_0.jpg
Resized and replaced: /content/GEN/00468_00_00468_00_0.jpg
Resized and replaced: /content/GEN/04611_00_04611_00_0.jpg
Resized and replaced: /content/GEN/04488_00_04488_00_0.jpg
Resized and replaced: /content/GEN/00158_00_00158_00_0.jpg
Resized and replaced: /content/GEN/00055_00_00055_00_0.jpg
Resized and replaced: /content/GEN/00008_00_00008_00_6.jpg
Resized and replaced: /content/GEN/04493_00_04493_00_0.jpg
Resized and replaced: /content/GEN/05857_00_05857_00_0.jpg
Resized and replaced: /content/GEN/00311_00_00311_00_0.jpg
Resized and replaced: /content/GEN/04615_00_04615_00_0.jpg
Resized and replaced: /content/GEN/06437_00_06437_00_0.j

### Import the required packages

In [ ]:
!pip install torchmetrics clean-fid torch-fidelity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 17.9 MB/s eta 0:00:00


In [ ]:
from PIL import Image
import os
from tqdm import tqdm
from cleanfid import fid
import torch
from torch.utils.data import Dataset, DataLoader
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from torchvision import transforms

### Generate Dataloaders from the images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class TestDataset(Dataset):
  def __init__(self, root_dir, transform=None):
    self.root_dir = root_dir
    self.transform = transform
    self.file_list = os.listdir(root_dir)

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, idx):
    file_name = self.file_list[idx]
    file_path = os.path.join(self.root_dir, file_name)

    img = Image.open(file_path)

    if self.transform:
        img = self.transform(img)

    return img

In [ ]:
def get_loader(root_dir, batch_size, num_workers):
  trans = transforms.Compose([
    transforms.ToTensor(),
  ])

  dataset = TestDataset(root_dir, trans)

  loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  return loader

### Prepare the metric classes

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model_is = InceptionScore(normalize=True).to(device)

ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

lpips = LearnedPerceptualImagePatchSimilarity(net_type='alex', normalize=True).to(device)

### Compute the metrics

In [ ]:
def compute_metrics(folder1, folder2, batch_size):
  # fid_score = fid.compute_fid(folder1, folder2, verbose=False)

  kid_score = fid.compute_kid(folder1, folder2, verbose=False)

  gt_loader = get_loader(folder1, batch_size, num_workers=2)
  test_loader = get_loader(folder2, batch_size, num_workers=2)

  for idx, (test_batch, gt_batch) in tqdm(enumerate(zip(gt_loader, test_loader)), total=len(gt_loader)):
    test_images = test_batch
    gt_images = gt_batch

    test_images = test_images.to(device)
    gt_images = gt_images.to(device)

    model_is.update(test_images)

    ssim.update(test_images, gt_images)

    lpips.update(test_images, gt_images)

  is_score, is_std = model_is.compute()

  ssim_score = ssim.compute()

  lpips_score = lpips.compute()

  return {
      # "fid":    fid_score,
      "kid":    kid_score,
      "is":     is_score.item(),
      "ssim":   ssim_score.item(),
      "lpips":  lpips_score.item(),
  }

In [ ]:
print("Computing the metrics for Generated-GT pairs...")
GEN_GT_metrics = compute_metrics(GEN_DIR, GT_DIR, BATCH_SIZE)
print(GEN_GT_metrics)

print("\n-----------------------------\n")

print("Computing the metrics for Distorted-GT pairs...")
DISTORTED_GT_metrics = compute_metrics(DISTORTED_DIR, GT_DIR, BATCH_SIZE)
print(DISTORTED_GT_metrics)

print("\n-----------------------------\n")

print("Improvement:")
improvement = {
    # 'fid':    DISTORTED_GT_metrics['fid'] - GEN_GT_metrics['fid'],      # Lower is better
    'kid':    DISTORTED_GT_metrics['kid'] - GEN_GT_metrics['kid'],      # Lower is better
    'is':     GEN_GT_metrics['is'] - DISTORTED_GT_metrics['is'],        # Higher is better
    'ssim':   GEN_GT_metrics['ssim'] - DISTORTED_GT_metrics['ssim'],    # Higher is better
    'lpips':  DISTORTED_GT_metrics['lpips'] - GEN_GT_metrics['lpips'],  # Lower is better
}
print(improvement)

Computing the metrics for Generated-GT pairs...


100%|██████████| 26/26 [00:00<00:00, 26.06it/s]


{'kid': -0.009359315842848605, 'is': 2.587279796600342, 'ssim': 0.610127329826355, 'lpips': 0.4599834680557251}

-----------------------------

Computing the metrics for Distorted-GT pairs...


100%|██████████| 30/30 [00:01<00:00, 26.16it/s]

{'kid': -0.007839400181094238, 'is': 2.834475040435791, 'ssim': 0.6053673624992371, 'lpips': 0.46821004152297974}

-----------------------------

Improvement:
{'kid': 0.0015199156617543667, 'is': -0.24719524383544922, 'ssim': 0.00475996732711792, 'lpips': 0.008226573467254639}
